In [27]:
import pandas as pd
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
# read in data:
traindata = pd.read_csv('final_train.csv')
test= pd.read_csv('final_test.csv')
# df = df.drop(columns='Unnamed: 0')
# df.reset_index(drop=True)

In [28]:
#Separate into train, and val
train=traindata[traindata['year']<=2005]
val=traindata[traindata['year']>2005]

In [29]:
train.shape, val.shape

((329, 64), (109, 64))

In [81]:
#Select target and features 
target='winner_bool'
features=train.columns.drop([target, 'winner', 'nconst_x','nconst_y','award', 'nominee', 'movie', 'year',
                             'year_film', 'year_film2', 'year_film3',
                              'tconst', 'primarytitle', 'startyear', 
                            'directors', 'writers', 'director','birthyear', 
                             'name', 'release_day',
                             'biography',
                           'drama', 'comedy', 'adventure', 'film-noir', 'mystery', 'action',
                           'crime', 'horror', 'family', 'animation', 'western', 'documentary',
                           'romance', 'fantasy', 'thriller', 'war', 'history', 'music', 'sport',
                           'musical', 'sci-fi', 'cpi_multiplier','opening_usa_day','release_day_of_month',
                             'release_month','world','runtimeminutes','budget','gross','n_globes'
                            ])

X_train=train[features]
X_val=val[features]
X_test=test[features]

y_train=train[target]
y_val=val[target]


In [ ]:
X_train.columns

<h2>Fit a Random Forest model </h2>

In [27]:
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer


mypipeline= make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_jobs=-1, random_state=42, max_depth=20, n_estimators=50)   
)
    
mypipeline.fit(X_train, y_train)
print(f'Accuracy score: {mypipeline.score(X_val, y_val)}')


Accuracy score: 0.8623853211009175


In [28]:
from joblib import dump
dump(mypipeline, 'mypipeline.joblib')

['mypipeline.joblib']

In [29]:
mypipeline.predict_proba(X_test)

array([[0.94, 0.06],
       [0.7 , 0.3 ],
       [0.9 , 0.1 ],
       [0.5 , 0.5 ],
       [0.96, 0.04],
       [0.94, 0.06],
       [0.82, 0.18],
       [0.52, 0.48],
       [0.78, 0.22]])

In [30]:
def predict(actor_in_a_leading_role_nomination,
       actor_in_a_supporting_role_nomination,
       actress_in_a_leading_role_nomination,
       actress_in_a_supporting_role_nomination, directing_nomination,
       cinematography_nomination,  averagerating,
       numvotes, dir_age, is_woman, rating, metascore, 
       opening_wk 
      ):
    df = pd.DataFrame(
        columns=['actor_in_a_leading_role_nomination',
           'actor_in_a_supporting_role_nomination',
           'actress_in_a_leading_role_nomination',
           'actress_in_a_supporting_role_nomination', 'directing_nomination',
           'cinematography_nomination',  'averagerating',
           'numvotes', 'dir_age', 'is_woman', 'rating', 'metascore', 
           'opening_wk'
          ],
        data=[[actor_in_a_leading_role_nomination,
           actor_in_a_supporting_role_nomination,
           actress_in_a_leading_role_nomination,
           actress_in_a_supporting_role_nomination, directing_nomination,
           cinematography_nomination, averagerating,
           numvotes, dir_age, is_woman, rating, metascore,
           opening_wk,
           ]]
    )
    y_pred=mypipeline.predict(df)[0]
    y_pred_proba=mypipeline.predict_proba(df)[0]
    return f'{y_pred_proba*100:.0f}% chance of {y_pred}'
   